# By-Band _g_-Point Reduction

# Dependencies

`numpy` is installed in the Python environment at NERSC (`module load python`), but `xarray` is not, so the user must install the package on their own. `PIPPATH` is the assumed location. This notebook depends heavily on `xarray`.

In [1]:
import os, sys, shutil

# "standard" install
import numpy as np

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
PATHS = ['common', PIPPATH]
for path in PATHS: sys.path.append(path)

# user must do `pip install xarray` on cori (or other NERSC machines)
import xarray as xa

# local module
import by_band_lib as BYBAND

# Static Inputs

In [2]:
# only do one domain or the other
DOLW = True
DOSW = not DOLW
NBANDS = 16 if DOLW else 14

# forcing scenario (0 is no forcing...need a more comprehensive list)
IFORCING = 0

# Paths

In [3]:
PROJECT = '/global/project/projectdirs/e3sm/pernak18/'
KFULLNC = '{}/reference_netCDF/g-point-reduce/'.format(PROJECT) + \
  'rrtmgp-data-lw-g256-2018-12-04.nc'
EXE = '{}/g-point-reduction/k-distribution-opt/rrtmgp_garand_atmos'.format(
    PROJECT)
GARAND = '{}/reference_netCDF/g-point-reduce/'.format(PROJECT) + \
  'lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc'
PATHS = [KFULLNC, EXE]
for PATH in PATHS: BYBAND.pathCheck(PATH)

CWD = os.getcwd()

# Band Splitting

Break up full _k_-distribution file into separate distributions for each band, then calculate the corresponding fluxes. This should only need to be run once.

After some clarifications from Robert (30-Nov-2020), I believe the plan of action is:

1. Perform _g_-point combination for 1 band at a time
2. Combine condensed _k_-distribution for a given band with the full distributions from the rest of the bands
3. Run RRTMGP on output from item 2
4. Compute cost function for broadband fluxes, not by-band

Original Plan of Action:

- [] We would create Nbands k-distribution files
- [] We’d use Python to drive the Fortran executable Nbands times to produce Nbands flux results
- [] The trial g-point combinations then loop over bands and the possible g-point combinations within each band, creating k-distribution and band-wise flux files for each possible combination
- [] The Python code assembles broadband fluxes from the band-wise flux files in order to compute the cost functions

In [4]:
# divide full k-distribution into subsets for each band
print('Band splitting commenced')
kFiles = BYBAND.kDistBandSplit(KFULLNC)
print('Band splitting completed')

# print('Full band flux calculations commenced')
# # think about multithreading this process

# fullBandFluxDir = 'full_band_flux'
# BYBAND.pathCheck(fullBandFluxDir, mkdir=True)
# fullBandFluxes = []
# for kFile in kFiles:
#     base = os.path.basename(kFile)
#     base = base.replace('coefficients', 'flux')
#     outFile = '{}/{}'.format(fullBandFluxDir, base)
#     #BYBAND.fluxCompute(kFile, outFile, fluxDir=fullBandFluxDir)
#     fullBandFluxes.append(outFile)
# # end kFile loop
# print('Full band flux calculations completed')

# small edit to flux file -- rename the `record` dimension

Band splitting commenced
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
Completed band_k_dist/coefficients_lw_band01.nc
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
Completed band_k_dist/coefficients_lw_band02.nc
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
Completed band_k_dist/coefficients_lw_band03.nc
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
Completed band_k_dist/coefficients_lw_band04.nc
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
Completed band_k_dist/coefficients_lw_band05.nc
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3)
(14, 9, 3

# _g_-Point Combining

Loop over bands and combine their respective _g_-points. For each combination in a band, compute fluxes, then concatenate these fluxes with the full-band fluxes from other bands, then compute broadband fluxes and corresponding cost function.

In [5]:
# loop over bands and instantiate a band optimization object
# 
bandObj = []
for iBand, kFile in enumerate(kFiles):
    band = iBand + 1

    iComb = 1
    kObj = BYBAND.kDistOptBand(kFile, band, DOLW, IFORCING, iComb)

    # combine g-points in band and generate corresponding netCDF
    kObj.gPointCombine()

    # generate input dictionaries for fluxComputePool()
    kObj.configParallel()
    continue

    # calculate fluxes corresponding to every g-point combination
    # break out of kDistOptBand object for 
    # computation of band fluxes in parallel
    BYBAND.fluxComputePool(kObj.fluxInputs)

    # replace original fluxes for band with modified one
    fluxesMod = list(fullBandFluxes)
    fluxesMod.remove(fullBandFluxes[iBand])
    fluxesMod.insert(iBand, outFile)

    # combine fluxes from modified band with unmodified bands
    #BYBAND.fluxCombine(kFilesMod)

    print('Band {} complete'.format(band))

    # cleanup
    shutil.rmtree(kObj.workDir)
# end kFile loop

band_k_dist/coefficients_lw_band01.nc
